In [2]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from IPython import get_ipython

# %%
get_ipython().system('pip install requests')
get_ipython().system('pip install dict2xml')
get_ipython().system('pip install lxml')
get_ipython().system('pip install googlesearch-python')

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for dict2xml: filename=dict2xml-1.7.0-py3-none-any.whl size=6326 sha256=ef5c51daa90a732b210805705f08e22df88703a32ed7056c513d649eeb0d3c18
  Stored in directory: /root/.cache/pip/wheels/99/2c/cb/8ffb54cf518a693d9157a071f5d428204fa3dc334a45b1abb1
Successfully built dict2xml
     |████████████████████████████████| 115 kB 5.0 MB/s 
     |████████████████████████████████| 61 kB 7.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [3]:
# %%
import requests
from lxml import html
from dict2xml import dict2xml
from googlesearch import search
from bs4 import BeautifulSoup
import re

In [10]:

# %%
output_file = "./whatever.xml"
starting_id = 97268 +1 # CHNAGE THIS TO LAST ID IN XML FILE


In [30]:
# %%
def get_unknown_fields(xpath_query):
    name, value = (t.strip() for t in tree.xpath(f"{xpath_query}/text()")[0].split(": "))
    name = name.lower().replace(" ", "_").replace("-", "_").split("(")[0]
    if not value:
        value = tree.xpath(f"{xpath_query}/a/text()")[0].strip()
    return name, value

In [33]:

# %%
def get_all_text(link):
    to_visit = []
    headers = {
    'User-Agent': 'My User Agent 1.0',
    'From': 'youremail@domain.com'  # This is another valid field
    }
    page = requests.get(link, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'})
    html_content = page.content
    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.find_all(text=True)
    output = ''
    for t in text:
        if t.parent.name not in ['script', 'em', 'style', 'option', 'span', '[document]', 'button']:
            output += '{} '.format(t)
    data = soup.findAll('div', attrs={'class':re.compile("article-content")})

    for div in data:
        links = div.findAll('a')
        for a in links:
          if "://" in a['href']:
            to_visit.append(a['href'])
          else:
            to_visit.append("https://ign.com"+a['href'])
    return output, set(to_visit)

def search_scrape(game_name):
    print(f"scraping for {game_name}")
    results =  [r for r in search(game_name, num_results=6) if "embed" not in r]
    output = ''
    to_visit = set()
    visited = []
    print(results)
    print(f"looking at 5 top results")
    for link in results[:4]:
      print(link)
      info, links = get_all_text(link)
      to_visit = set.union(to_visit, links)
      visited.append(link)
      output += info
    
    print(f"looking at first layer of links")
    to_visit_2 = set()
    for link in to_visit:
      if link in visited:
        continue
      info, links = get_all_text(link)
      to_visit_2 = set.union(to_visit_2, links)
      visited.append(link)
      output += info
    
    print(f"looking at second layer of links")
    for link in to_visit_2:
      if link in visited:
        continue
      info, _ = get_all_text(link)
      visited.append(link)
      output += info
    
    print(f"scraping finished for {game_name}")
    return output
  


In [34]:

# %%
# scrapes data from "https://thegamesdb.net/game.php?id={i}" and appends to output_file.xml
with open(output_file, "a", encoding="utf-8") as f:
    f.write("<GAMES>") 
    for i in range(0, 10):
        url = f"https://thegamesdb.net/game.php?id={i}"
        page = requests.get(url)
        tree = html.fromstring(page.text)

        try:
            game_name = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[1]/h1/text()")[0].strip()
            description = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[2]/p[1]/text()")[0].strip()
        except IndexError: continue

        gamesdb_link = url
        esrb_rating = ""
        genres = ""

        for j in range(2,6):
            try:
                name, value = tree.xpath(f"/html/body/div/div[2]/div[2]/div[1]/div/div/div[2]/p[{j}]/text()")[0].split(": ")
                if value:
                        esrb_rating = value if "esrb" in name.lower() else esrb_rating
                        genres = value if "genre" in name.lower() else genres
            except IndexError: continue

        data = {
                    "game_id": i,
                    "game_name": game_name,
                    "description": description,
                    "esrb_rating": esrb_rating,
                    "genres": genres, 
                    "gamesdb_link": gamesdb_link,
                    "ign": search_scrape(game_name)
                }

        for k in range(2,10):
            try:
                xpath_query = f"/html/body/div/div[2]/div[1]/div/div/div/div[2]/p[{k}]"
                name, value = get_unknown_fields(xpath_query)
                data[name] = value
            except IndexError: continue

        xml = dict2xml(data, indent=False, wrap="game")
        f.write(xml)
        print(xml, sep="\n----\n")
        # print(i)
    f.write("</GAMES>")


# %%
print("done")

Streaming output truncated to the last 5000 lines.
 
  banner  
 
 
 
 
 
 
 
 
 
   
 EXPLORE THE COLLECTION 
 
 
 
 
 
 
  end banner  
  main content pushed down on homepage  
 
  begin main content  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 SHOP NOW 
 
 
 
 SHOP TUDOR 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
  end main content  
  begin footer  
 
 
 
 
 Shop 
 
 Engagement Jewelry Designers Watches 
 
 
 
 Explore Tapper's 
 
 About Us Careers 
 Events 
 
 
 
 Help 
 
 Customer Care Contact 
 
 
 
  Begin MailChimp Signup Form  
 
 
 
 
 Sign up for exclusive offers and updates. 
 
 
 
 
 
 
 End mc_embed_signup 
 
 
 
 
 
 
 
 
 
 
 
 
                ©2022. Tapper's. All Rights Reserved. 
 Privacy    |  
                 Accessibility Policy    |  
                 Terms    |  
                 Sitemap 
 
 
 
 
 
 
 
 
 
  end footer  
 
 
 
 
 
 
 
 
 
 Home 
 Rolex 
 Engagement 
 Jewelry 
 David Yurman 
 Designers 
 Watches 
 Sale 
 
 
 About Us 
 Make Appointment 
 Cu